In [1]:
#import get to call a get request on the site
import requests

#import to manipulate arrays with numpy
import numpy as np

#import to create, clean, and parse data frames with pandas
import pandas as pd

#import to enable datascraping
from bs4 import BeautifulSoup

#import to set up 'sleep' to wait between page loads
import time

# import Mongo so our webscraper dumps its scraped data without losing it
from pymongo import MongoClient
import pymongo

#import to make that html readable
import pprint

#import regular expressions operations
import re

#import to get the universe in balance
import random

#import so we can do some heavy stats work
import scipy as sp
from scipy.stats import binom
import scipy.stats as stats

#import to access certain plotting features
import seaborn as sns

#import because we need its program functions
import math

#import because we need to plot and make it pretty
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
plt.style.use('ggplot')

In [2]:
client = MongoClient('localhost', 27017)
db = client['craigslist_motorcycles']
post_html = db['motorcycle_posts']

In [3]:
pprint.pprint(post_html.find()[5]['html'])

('<!DOCTYPE html>\n'
 '<html class="no-js">\n'
 '<head>\n'
 '    \n'
 '\t<meta charset="UTF-8">\n'
 '\t<meta http-equiv="X-UA-Compatible" content="IE=Edge">\n'
 '\t<meta name="viewport" content="width=device-width,initial-scale=1">\n'
 '\t<meta property="og:site_name" content="craigslist">\n'
 '\t<meta name="twitter:card" content="preview">\n'
 '\t<meta property="og:title" content="2016 Harley Davidson Roadster Sportster '
 'for Sale -...">\n'
 '\t<meta name="description" content="2016 Harley-Davidson Sportster Roadster '
 '(EVO 1202cc) in excellent condition for sale $6,999 OBO....">\n'
 '\t<meta property="og:description" content="2016 Harley-Davidson Sportster '
 'Roadster (EVO 1202cc) in excellent condition for sale $6,999 OBO....">\n'
 '\t<meta property="og:image" '
 'content="https://images.craigslist.org/00707_gaqoQHqMFur_600x450.jpg">\n'
 '\t<meta property="og:url" '
 'content="https://auburn.craigslist.org/mcy/d/auburn-2016-harley-davidson-roadster/7114312352.html">\n'
 '\t<met

In [48]:
rad = db['motorcycle_posts'].find()[0:1000]
for i in rad:    
    print (i['_id'])

https://auburn.craigslist.org/mcy/d/auburn-university-2016-honda-cbr300r/7120429265.html
https://auburn.craigslist.org/mcd/d/fayetteville-bad-credit-no-credit-ok-we/7119776172.html
https://auburn.craigslist.org/mcy/d/montgomery-2017-yamaha-fz-07-very-low/7118845773.html
https://auburn.craigslist.org/mcd/d/haleyville-2010-yellow-goldwing-gl1800/7116263343.html
https://auburn.craigslist.org/mcy/d/auburn-2006-honda-xr650l-price-drop/7114333691.html
https://auburn.craigslist.org/mcy/d/auburn-2016-harley-davidson-roadster/7114312352.html
https://auburn.craigslist.org/mcy/d/opelika-2017-concours-1400/7111191074.html
https://bham.craigslist.org/mcy/d/tuscaloosa-2008-harley-davidson/7122367787.html
https://bham.craigslist.org/mcy/d/springville-2007-yamaha-star-1300/7122326387.html
https://bham.craigslist.org/mcy/d/birmingham-1993-honda-goldwing-gl1500/7122190883.html
https://bham.craigslist.org/mcy/d/childersburg-ktm-50-sx-pro-senior-lc/7121832377.html
https://bham.craigslist.org/mcy/d/pelham-

In [21]:
def motorcycle_post_parser():
    client = MongoClient('localhost', 27017)
    db = client['craigslist_motorcycles']
    post_html = db['motorcycle_posts'].find()[0:1000]
    
    title = []
    price = []
    neighborhood = []
    description = []
    url = []
    #attributes
    model = []
    vin = []
    condition = []
    cryptocurrency = []
    delivery = []
    engine_size = []
    fuel_type = []
    mileage = []
    paint_color = []
    street_legal = []
    title_status = []
    transmission = []
    bike_type = []

    iterations = 0

    for _ in post_html:         
        
        # attributes key words to parse through attributes
        vin_ = 'VIN:'
        condition_ = 'condition:'
        cryptocurrency_ = 'cryptocurrency'
        delivery_ = 'delivery'
        engine_size_ = 'engine displacement'
        fuel_type_ = 'fuel:'
        mileage_ = 'odometer:'
        paint_color_ = 'paint color:'
        street_legal_ = 'street legal'
        title_status_ = 'title status:'
        transmission_ = 'transmission:'
        bike_type_ = 'type:'
        
         
        
        response = _['html']
        soup = BeautifulSoup(response, 'html.parser')

        try:
            post_title = soup.find('span', id = 'titletextonly')
            title.append(post_title.text)
        except:
            title.append(str(post_title))
        else:
            title.append('')
        
        try:
            post_price = soup.find('span', class_ = 'price').text
            post_price.lstrip('$')
            price.append(post_price)
        except:
            price.append('')
        
        try:
            post_neighborhood = soup.find('small')
            neighborhood.append(post_neighborhood.text)
        except:
            neighborhood.append('')
        
        try:
            post_description = soup.find('section', id = 'postingbody')
            description.append(post_description.text)
        except:
            description.append(post_description)
        else:
            description.append('')
        
        url.append(_['_id'])
        
        post_attributes = str(soup.find_all('p', attrs = {'class': 'attrgroup'}))
        post_attributes = post_attributes.replace(post_attributes, str(post_attributes.replace('''[<p class="attrgroup">\n<span><b>''', '').replace('''</b></span>\n<br/>\n</p>, ''','').replace('''<p class="attrgroup">\n<span>''', ',').replace(''' <b>''', '').replace('''</b></span>\n<br/>\n<span>''', ',').replace('''</span>\n<br/>\n<span>''', ',').replace('''</b></span>\n<br/>\n</p>]''', '').replace('''<p class="attrgroup">\n<span class="otherpostings">\n<a href="//auburn.craigslist.org/search/sss?userid=193613506">\nmore ads  by this user        </a>\n</span>\n</p>]''', '')))
                    
        bike_attributes = (post_attributes.split(','))

        vin_res = [i for i in bike_attributes if vin_ in i]
        condition_res = [i for i in bike_attributes if condition_ in i]
        cryptocurrency_res = [i for i in bike_attributes if cryptocurrency_ in i]
        delivery_res = [i for i in bike_attributes if delivery_ in i]
        engine_size_res = [i for i in bike_attributes if engine_size_ in i]
        fuel_type_res = [i for i in bike_attributes if fuel_type_ in i]
        mileage_res = [i for i in bike_attributes if mileage_ in i]
        paint_color_res = [i for i in bike_attributes if paint_color_ in i]
        street_legal_res = [i for i in bike_attributes if street_legal_ in i]
        title_status_res = [i for i in bike_attributes if title_status_ in i]
        transmission_res = [i for i in bike_attributes if transmission_ in i]
        bike_type_res = [i for i in bike_attributes if bike_type_ in i]

        try:
            model.append(bike_attributes[0])
        except:
            model.append('')
                
        try:
            vin.append(vin_res[0])
        except:
            vin.append('')
            
        try:
            condition.append(condition_res[0])
        except:
            condition.append('')
               
        try:
            cryptocurrency.append((cryptocurrency_res[0]))
        except:
            cryptocurrency.append('')
        
        try:
            delivery.append(delivery_res[0])
        except:
            delivery.append('')
                
        try:
            engine_size.append(engine_size_res[0])
        except:
            engine_size.append('')
            
        try:
            fuel_type.append(fuel_type_res[0])
        except:
            fuel_type.append('')
            
        try:
            mileage.append(mileage_res[0])
        except:
            mileage.append('')
                    
        try:
            paint_color.append(paint_color_res[0])
        except:
            paint_color.append('')
                 
        try:
            street_legal.append(street_legal_res[0])
        except:
            street_legal.append('')
    
        try:
            title_status.append(title_status_res[0])
        except:
            title_status.append('')
                
        try:
            transmission.append(transmission_res[0])
        except:
            transmission.append('')
            
        try:
            bike_type.append(bike_type_res[0])
        except:
            bike_type.append('')
                
        iterations += 1

        if iterations%100 == 0:
            print(str(iterations) + ' parsed succesfully')

    return title, price, neighborhood, attributes, description, url, model, vin, condition, cryptocurrency, delivery, engine_size, fuel_type, mileage, paint_color, street_legal, title_status, transmission, bike_type

In [22]:
title, price, neighborhood, attributes, description, url, model, vin, condition, cryptocurrency, delivery, engine_size, fuel_type, mileage, paint_color, street_legal, title_status, transmission, bike_type = motorcycle_post_parser()

100 parsed succesfully
200 parsed succesfully
300 parsed succesfully
400 parsed succesfully
500 parsed succesfully
600 parsed succesfully
700 parsed succesfully
800 parsed succesfully
900 parsed succesfully
1000 parsed succesfully


In [23]:
len(title), len(price), len(neighborhood), len(attributes), len(description), len(url), len(model), len(vin), len(condition), len(cryptocurrency), len(delivery), len(engine_size), len(fuel_type), len(mileage), len(paint_color), len(street_legal), len(title_status), len(transmission), len(bike_type)

(1999,
 1000,
 1000,
 1000,
 1999,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000)

In [18]:
model

['2016 honda cbr300r',
 ' all bikes',
 '2017 Yamaha FZ-07',
 '2010 Honda GL1800 CSC',
 '2006 honda xr650l',
 '2016 harley-davidson roadster',
 '2017 2017 kawasaki concours 1400zg',
 '2008 harley davidson heritage classic',
 '2007 yamaha v star 1300',
 '1993 Handa Goldwing',
 '2003 ktm 50 sx',
 '2017 2017 Kawasaki Vulcan Vaquero abs',
 '2011 yamaha fz6r',
 '2013 Harley Davidson Street Glide',
 ' harley davidson',
 '2009 kawasaki',
 '1965 Harley Davidson Electra Glide',
 '2010 harley davidson ultra classic',
 '2010 harley davidson ultra classic',
 '2006 2006 ninja ZX14',
 '2010 kawasaki zx6r',
 '2014 Harley Tri Glide',
 '2014 Harley Tri Glide',
 '2014 BMK K1600 GTL',
 '2017 Suzuki GSX-R750',
 '2008 harley davidson',
 '2019 indian scout',
 ' ducati multistrada',
 '2006 harley davidson',
 '2015 2015 Honda PCX 150 cc',
 '2015 Harley Davidson XG500',
 ' Harley-Davidson',
 '1983 honda goldwing interstate',
 ' harley davidson fdfx',
 '2009 suzuki vstrom 650',
 '2003 O3 Harley',
 '2002 harley d

In [ ]:
title = []

for _ in title15:
    for __ in (_):
#         __ = str(_[idx]).split('''>''')[0]
#         __ = str(_[idx]).split('''</''')[-2]
        title.append(__)

price = []

for _ in price15:
    for __ in _:
#         __ = str(_[idx]).split('''>''')[1]
#         __ = str(_[idx]).split('''</''')[-2]
        price.append(__)

neighborhood = []

for _ in neighborhood15:
    for __ in _:
#          __ = str(_[idx]).split('''(''')[1]
#         __ = str(_[idx]).split(''')''')[-2]
        neighborhood.append(__)

attributes = []

for _ in attributes15:
    for __ in _:
        attributes.append(__)

description = []

for _ in description15:
    for __ in _:
        description.append(__)

city = []

for _ in city15:
    for __ in _:
        city.append(__)